# Compute residual sidewalk width

This notebook demonstrates how to compute the residual sidewalk width based on obstacles extracted from MLS points clouds. Using two scans of the same area we apply change detection to extract static obstacles above the sidewalk. These are then used to compute the residual width.

Input (for each tile):
- Two pointcloud scans
- Elevation data (unless gronund points are labaled)
- Sidewalk polygons

Output:
- Polygons of static obstacles above the sidewalk
- Paths traversing the sidewalk annotated with minimal width per meter

In [ ]:
# Add project src to path.
import set_path

import numpy as np
import pathlib
from tqdm.notebook import tqdm

import upcp.fusion as fusion
import upcp.utils.ahn_utils as ahn_utils
import upcp.utils.bgt_utils as bgt_utils
import upcp.utils.log_utils as log_utils
import upcp.utils.las_utils as las_utils
from upcp.labels import Labels

# Local imports
import upc_sw.sw_utils as sw_utils

In [ ]:
# Data folders.
ahn_data_folder = '../datasets/ahn/'
bgt_data_file = '../datasets/bgt/bgt_voetpad.csv'
pc_data_folder = '../datasets/pointclouds/'
pc_file_prefix = 'processed'

ground_labels = [Labels.GROUND, Labels.ROAD]

In [ ]:
# AHN elevation reader.
ahn_reader = ahn_utils.NPZReader(data_folder=ahn_data_folder, caching=False)

# Sidewalk polygon reader.
sw_poly_reader = bgt_utils.BGTPolyReader(bgt_file=bgt_data_file)

# Ground fuser using pre-processed AHN data.
ground_fuser = fusion.AHNFuser(Labels.GROUND, ahn_reader=ahn_reader,
                               target='ground', epsilon=0.2,
                               params={'bottom': 0., 'buffer': 0.02})

In [ ]:
# Create folders for obstacle files.
for run in ['run1', 'run2']:
    new_path = f'{pc_data_folder}obstacles_{run}'
    pathlib.Path(new_path).mkdir(parents=True, exist_ok=True)

In [ ]:
# Get a list of all tilecodes for which we have two runs.
all_tiles = (las_utils.get_tilecodes_from_folder(f'{pc_data_folder}run1/')
             .union(las_utils.get_tilecodes_from_folder(f'{pc_data_folder}run2/')))

In [ ]:
tile_tqdm = tqdm(all_tiles, unit='tile', smoothing=0)

for tilecode in tile_tqdm:
    tile_tqdm.set_postfix_str(tilecode)
    print(f'Processing tile {tilecode}...')
    for run in ['run1', 'run2']:
        file = f'{pc_data_folder}{run}/{pc_file_prefix}_{tilecode}.laz'
        
        # Load pointcloud data.
        points, labels = sw_utils.read_las(file, extra_val='label')
        obstacle_mask = np.zeros((len(points),), dtype=bool)
        
        # Load ground points.
        if np.count_nonzero(labels) > 0:
            print('Using labels found in pointcloud file.')
            ground_mask = sw_utils.create_label_mask(labels, target_labels=ground_labels)
        else:
            mask = np.ones((len(points),), dtype=bool)
            ground_mask = ground_fuser.get_label_mask(points, labels, mask, tilecode)
        
        # Extract points aboves sidewalk.
        mask_ids = np.where(~ground_mask)[0]
        sw_mask = sw_utils.sidewalk_clip(
                                points[~ground_mask], tilecode, sw_poly_reader=sw_poly_reader,
                                ahn_reader=ahn_reader, max_height=2.0)
        obstacle_mask[mask_ids] = sw_mask
        
        # Save the new point cloud
        out_file = f'{pc_data_folder}obstacles_{run}/obst_{tilecode}.laz'
        sw_utils.write_las(points[obstacle_mask], out_file, values=labels[obstacle_mask])